# Spotify data api application

## Imports

In [17]:
#!pip install spotipy
#!pip install bs4
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

#BillbordTop#100Songs

## Spotify api credentials

In [18]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id="3a2f65eb20164b24b59b7fdd6876bb51", client_secret="9f6d3a070cdf4eb8adda14a794d60469")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Fetch data from playlist or single song

In [19]:
playlist_link = "https://open.spotify.com/playlist/2GxgTnPCRkKl6sTX1INdAw?si=74fe4ed6868b408a"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [3]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    
    print(track["track"])
    #Printin details
    #print(track_name,"\n", 
    #      artist_name,"\n",
    #      #sp.audio_features(track_uri)[0], "\n", 
    #      track_uri
    #     )
    

NameError: name 'sp' is not defined

## Fetching song metrics

In [21]:
sp.audio_features(track_uri)[0]

{'danceability': 0.722,
 'energy': 0.745,
 'key': 7,
 'loudness': -4.164,
 'mode': 1,
 'speechiness': 0.256,
 'acousticness': 0.247,
 'instrumentalness': 0,
 'liveness': 0.419,
 'valence': 0.52,
 'tempo': 77.029,
 'type': 'audio_features',
 'id': '5UlnuulVAVmmesw4VzqHdG',
 'uri': 'spotify:track:5UlnuulVAVmmesw4VzqHdG',
 'track_href': 'https://api.spotify.com/v1/tracks/5UlnuulVAVmmesw4VzqHdG',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5UlnuulVAVmmesw4VzqHdG',
 'duration_ms': 173381,
 'time_signature': 4}

## Connecting Spotify API with Genius Lyrics

In [22]:
def get_album_tracks(uri_info):
    uri = []
    track = []
    duration = []
    explicit = []
    track_number = []
    one = sp.album_tracks(uri_info, limit=50, offset=0, market='US')
    df1 = pd.DataFrame(one)
    
    for i, x in df1['items'].items():
        uri.append(x['uri'])
        track.append(x['name'])
        duration.append(x['duration_ms'])
        explicit.append(x['explicit'])
        track_number.append(x['track_number'])
    
    df2 = pd.DataFrame({
    'uri':uri,
    'track':track,
    'duration_ms':duration,
    'explicit':explicit,
    'track_number':track_number})
    
    return df2

In [23]:
def get_track_info(df):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    for i in df['uri']:
        for x in sp.audio_features(tracks=[i]):
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
            
    df2 = pd.DataFrame({
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo})
    
    return df2

In [24]:
def merge_frames(df1, df2):
    df3 = df1.merge(df2, left_index= True, right_index= True)
    return df3

In [25]:
print (get_album_tracks("https://open.spotify.com/album/3mH6qwIy9crq0I9YQbOuDf?si=R7e_EQ8eS5-oRRljQuR9lw"))

                                     uri           track  duration_ms  \
0   spotify:track:19YKaevk2bce4odJkP5L22           Nikes       314075   
1   spotify:track:2ZWlPOoWh0626oTaHrnl2a             Ivy       249191   
2   spotify:track:3xKsf9qdS1CyvXSMEid6g8    Pink + White       184516   
3   spotify:track:1IdZvb2XK8NmsovQlV6CPn     Be Yourself        86943   
4   spotify:track:35xSkNIXi504fcEwz9USRB            Solo       257380   
5   spotify:track:4xR3MAscflQ262kMeiKshQ      Skyline To       184600   
6   spotify:track:5GUYJTQap5F3RDQiCOJhrS    Self Control       249667   
7   spotify:track:2JUqYobT8NvARdPmc4ES2x        Good Guy        66786   
8   spotify:track:7eqoqGkKwgOaWNNHx90uEZ          Nights       307151   
9   spotify:track:2qtoRFCOEL1gRn5q9DJC7F  Solo (Reprise)        78742   
10  spotify:track:17yrCsl1Ai6CZLBmGj6d6p    Pretty Sweet       158478   
11  spotify:track:73D5hBkiVZkopZrmBZTBWf  Facebook Story        68588   
12  spotify:track:1VZLEW5ZfcAbKZ94XQiSZF    Close T

In [26]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['track']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1

In [27]:
print(scrape_lyrics('frank ocean', 'Nikes'))

None


In [28]:
blonde_df1_tracks = get_album_tracks('spotify:album:3mH6qwIy9crq0I9YQbOuDf')
blonde_df2_metadata = get_track_info(blonde_df1_tracks)
df1 = merge_frames(blonde_df1_tracks, blonde_df2_metadata)
lyrics_onto_frame(df1, 'frank ocean')

,uri,track,duration_ms,explicit,track_number,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,spotify:track:19YKaevk2bce4odJkP5L22,Nikes,314075,True,1,0.543,0.3940,8,-9.637,0.0864,0.695,0.000239,0.1080,0.1180,137.811,NaN
1,spotify:track:2ZWlPOoWh0626oTaHrnl2a,Ivy,249191,True,2,0.567,0.3880,9,-9.579,0.0384,0.782,0.000309,0.2480,0.4520,116.362,NaN
2,spotify:track:3xKsf9qdS1CyvXSMEid6g8,Pink + White,184516,False,3,0.544,0.5520,9,-7.450,0.0991,0.670,0.000046,0.4150,0.5540,159.738,NaN
3,spotify:track:1IdZvb2XK8NmsovQlV6CPn,Be Yourself,86943,False,4,0.657,0.4310,7,-11.147,0.5410,0.985,0.000000,0.1490,0.9350,83.509,NaN
4,spotify:track:35xSkNIXi504fcEwz9USRB,Solo,257380,True,5,0.444,0.2870,5,-10.631,0.0375,0.849,0.000003,0.1610,0.2460,124.027,NaN
5,spotify:track:4xR3MAscflQ262kMeiKshQ,Skyline To,184600,True,6,0.423,0.2940,5,-13.673,0.0593,0.882,0.000005,0.0981,0.4010,129.146,NaN
6,spotify:track:5GUYJTQap5F3RDQiCOJhrS,Self Control,249667,False,7,0.572,0.2090,8,-10.413,0.0313,0.765,0.000000,0.3560,0.4460,80.069,NaN
7,spotify:track:2JUqYobT8NvARdPmc4ES2x,Good Guy,66786,False,8,0.513,0.1310,0,-12.787,0.4670,0.983,0.004140,0.1200,0.8140,105.808,NaN
8,spotify:track:7eqoqGkKwgOaWNNHx90uEZ,Nights,307151,True,9,0.466,0.5480,5,-9.362,0.1180,0.420,0.000001,0.1130,0.4230,89.815,NaN
9,spotify:track:2qtoRFCOEL1gRn5q9DJC7F,Solo (Reprise),78742,False,10,0.541,0.7470,7,-6.811,0.1620,0.404,0.000000,0.0860,0.6040,83.286,NaN


In [30]:
import requests

url = "https://billboard-api2.p.rapidapi.com/billboard-200"

querystring = {"range":"1-10","date":"2019-05-11"}

headers = {
	"X-RapidAPI-Host": "billboard-api2.p.rapidapi.com",
	"X-RapidAPI-Key": "597c838e95msh4535f09d502ea23p1da18cjsn1ea96f8f58cc"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}
